In [1]:
import numpy as np 
import pandas as pd 

import torch
import torch.nn as nn 

import plotly.express as px 
import plotly.io as pio 

pio.templates.default = "plotly_white"

In [2]:
df = pd.read_excel("data/load.xlsx", header=None)
df.rename(columns = {0 : "y"}, inplace = True)
df.head()


,y
0,447
1,435
2,451
3,442
4,444


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4380 entries, 0 to 4379
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   y       4380 non-null   int64
dtypes: int64(1)
memory usage: 34.3 KB


## Plot time series

In [4]:
p = px.line(df, y = "y")
p.update_traces(mode = "lines+markers", marker = dict(size = 3, color = "red"))
p.show()

In [6]:
px.histogram(df, "y")

## Prepare Data

In [23]:
y = df.y.values.astype(float)

test_size = int(0.3*len(y))

train_set = y[:-test_size]
test_set = y[-test_size:]

print(f"train size : {train_set.shape}")
print(f"test size : {test_set.shape}")

train size : (3066,)
test size : (1314,)


## Normalise Data

In [25]:
from sklearn.preprocessing import MinMaxScaler

In [27]:
scaler = MinMaxScaler(feature_range = (-1,1))
train_norm = scaler.fit_transform(train_set.reshape(-1,1))
print(f"train norm -- min : {train_norm.min()}, max : {train_norm.max()}")
print(f"train norm first five : \n{train_norm[:5]}")

train norm -- min : -1.0, max : 1.0
train norm first five : 
[[-0.74409449]
 [-0.79133858]
 [-0.72834646]
 [-0.76377953]
 [-0.75590551]]


## Sliding Window

In [114]:
train_norm = torch.FloatTensor(train_norm).view(-1)
print(f"train_norm shape : {train_norm.shape}")

window_size = 25

def input_data(seq,ws):
    out = []
    L = len(seq)
    for i in range(L-ws):
        window = seq[i:i+ws]
        label = seq[i+ws:i+ws+1]
        out.append((window, label))
    return out

print("---- Test Sliding window function ---- ")
input_data(list(range(0,10)), 3)

train_norm shape : torch.Size([3066])
---- Test Sliding window function ---- 


[([0, 1, 2], [3]),
 ([1, 2, 3], [4]),
 ([2, 3, 4], [5]),
 ([3, 4, 5], [6]),
 ([4, 5, 6], [7]),
 ([5, 6, 7], [8]),
 ([6, 7, 8], [9])]

In [115]:
train_data = input_data(train_norm, window_size)

## LSTM Model

In [131]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=100, output_size=1):
        super().__init__()

        self.hidden_size = hidden_size

        # Add a LSTM Layer
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size)

        # FCC
        self.linear = nn.Linear(in_features=hidden_size, out_features=output_size)

        # Hidden state - initialise
        # inputs : (sequence_length = 1, batch_size = 1, input_size/neurons = 100) 
        self.h0 = torch.zeros(1,1,hidden_size)
        self.c0 = torch.zeros(1,1,hidden_size)

    def forward(self,seq):
        # ---Input shapes ---
        #seq.view(len(seq),1,-1) : (150,1,1)
        # ---output shapes---
        #lstm_out shape : (150,1,100) --(seq length, if bidirectional 2 else 1, hidden_size/neurons)
        # h0/c0 shape shape : (1,1,100) --(bi-directional * num layers, batch_size, hidden_size/neurons)
        lstm_out, (h0,c0) = self.lstm(seq.view(len(seq), 1, -1), (self.h0, self.c0)) 
        # ---Input shapes ---
        # lstm_out.view(len(seq), -1) : (150,100)
        # ---Output shapes---
        # pred shape : (150, 1)
        pred = self.linear(lstm_out.view(len(seq), -1))
        return pred[-1]

In [134]:
window_size = 300
torch.manual_seed(101)
model = LSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [135]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f"{item:>6}")
    print(f"_______\n{sum(params):>6}")

count_parameters(model)

   400
 40000
   400
   400
   100
     1
_______
 41301


## Train

In [119]:
import time

In [136]:
epochs = 100
start_time = time.time()

for epoch in range(epochs):
    # extract the sequence and labels from training data
    
    for seq, y_train in train_data: #seq shape (150,) | y_train shape (1,)
        optimizer.zero_grad()
        h0 = torch.zeros(1,1,model.hidden_size)
        c0 = torch.zeros(1,1,model.hidden_size)
        y_hat = model.forward(seq)
        loss = criterion(y_hat,y_train)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch : {epoch+1:2} Loss : {loss.item():10.8f}")
print(f"\nDuration : {time.time() - start_time:.0f} seconds")
        

Epoch :  1 Loss : 0.00001832
Epoch :  2 Loss : 0.01557388
Epoch :  3 Loss : 0.01290543
Epoch :  4 Loss : 0.01215745
Epoch :  5 Loss : 0.05516334
Epoch :  6 Loss : 0.05524185
Epoch :  7 Loss : 0.03903681
Epoch :  8 Loss : 0.01732028
Epoch :  9 Loss : 0.04169080
Epoch : 10 Loss : 0.02756972
Epoch : 11 Loss : 0.09347088
Epoch : 12 Loss : 0.03538002
Epoch : 13 Loss : 0.05754430
Epoch : 14 Loss : 0.05462854
Epoch : 15 Loss : 0.05852445
Epoch : 16 Loss : 0.07067718
Epoch : 17 Loss : 0.01852107
Epoch : 18 Loss : 0.01497281
Epoch : 19 Loss : 0.04887091
Epoch : 20 Loss : 0.01144192
Epoch : 21 Loss : 0.03790995
Epoch : 22 Loss : 0.02379044
Epoch : 23 Loss : 0.05345014
Epoch : 24 Loss : 0.09581836
Epoch : 25 Loss : 0.06802356
Epoch : 26 Loss : 0.05157943
Epoch : 27 Loss : 0.02901377
Epoch : 28 Loss : 0.01622709
Epoch : 29 Loss : 0.01032996
Epoch : 30 Loss : 0.04997264
Epoch : 31 Loss : 0.01891053
Epoch : 32 Loss : 0.01858929
Epoch : 33 Loss : 0.06933194
Epoch : 34 Loss : 0.02632534
Epoch : 35 Los

# Prediction

In [137]:
future = test_size

# Add the last window from train set, which will be used to predict the testing window
preds = train_norm[-window_size:].tolist() # note preds include the last 150 datapoints from the trainset

# set the model to evaluation mode
model.eval()

for i in range(future):
    seq =torch.FloatTensor(preds[-window_size:])
    with torch.no_grad():
        model.h0 = torch.zeros(1,1,model.hidden_size)
        model.c0 = torch.zeros(1,1,model.hidden_size)
        preds.append(model.forward(seq).item())


In [129]:
from sklearn.metrics import mean_squared_error

predictions = scaler.inverse_transform(np.array(preds[window_size:]).reshape(-1,1)).flatten()
print(f"MSE : {mean_squared_error(y_pred=predictions,y_true=test_set)}")


MSE : 32638.507331759974


In [139]:
p1 = px.line(y = test_set)
p1.add_scatter(y = predictions, name = "prediction")
p1.show()

# Method 2 

## Prophet